# 将scRNA-seq和scATAC-seq原始fastq文件转换为gene-cell matrix和peak-cell matrix

以ASTAR-seq的[GSE113418](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE113418)为例, 由于给出的文件为每个细胞单独测序的fastq，所以使用脚本循环进行处理

## scATAC-seq fastq处理

### 下载原始文件、比对

运行脚本

In [ ]:
#!/bin/bash

for line in `cat SRR_Acc_List.txt`    # SRR_Acc_List文件包含了每个细胞初始数据的SRR号
do
    # 下载SRA 
    prefetch $line
    cd $line
    # 将SRA文件拆解为fastq文件
    fasterq-dump -p --include-technical -S -e 10 -O ./ ${line}.sra
    rm ${line}.sra

    # 比对
    bwa mem -t 8 /sibcb1/chenluonanlab6/zhangchuanchao2/data/Homo_sapiens/UCSC/hg38/Sequence/BWAIndex/genome.fa ${line}_1.fastq ${line}_2.fastq | samtools view -b - > ${line}.bam
    rm ${line}_1.fastq
    rm ${line}_2.fastq

    # 排序和加索引
    samtools sort -n ${line}.bam -o ${line}.sort.bam
    rm ${line}.bam

    # peak calling
    Genrich -t ${line}.sort.bam -o ${line}.narrowPeak -j -r -q 0.05
    cd ..
done

![narrowPeak文件](img/narrowpeak.png)

### 从narrowPeak文件到peak-cell matrix

运行脚本，multiIntersectBed可以得到输入样本间存在的共同位置，以及在该位置上的peak数

In [ ]:
#!/bin/bash

files=''
for line in `cat SRR_Acc_List.txt`
do
    # 首先用merge转换为只包含染色体及起始位置的bed文件
    bedtools merge -i ${line}/${line}.narrowPeak > test/${line}.bed
    cd test
    # 去除常染色体外的片段
    sed '/chrM\|chrUn\|chr[0-9]\{1,2\}_/d' ${line}.bed > ${line}.new.bed
    rm ${line}.bed
    # 为后续multiIntersectBed编写命令中需要的参数
    files=${files}" "${line}".new.bed"
    cd ..
done

cd test
# 使用multiIntersectBed将所有的细胞的bed文件整合为peak-cell matrix
multiIntersectBed -i ${files} -header > test.bed

multiIntersectBed示例结果：

![multiIntersectBed示例结果](img/sample.png)

由于multiIntersectBed产生的文件中还会包含存在重合的样本数量、重合的样本名，而且可能存在上下两个染色体位置彼此相连，所以我们再通过bedtools merge进行处理

[merge参数设置参考](https://www.biostars.org/p/103076/)：在merge的过程中，相连的染色体位置会整合在一起，所以相应的peak数应该相加，使用-c设置需要相加的列，-o设置对每列使用的方法（即sum

In [ ]:
#!/bin/bash

columns=''
nsum="sum"
for i in {6..389};
do
    if [ "$i" -eq 6 ];then
    nsum="sum"
    columns="6"
    else nsum=${nsum}",sum"
    columns=${columns}","${i}
    fi
done

bedtools merge -i test.bed -c ${columns} -o ${nsum} > test0.bed

由于我们最终需要的matrix如下图所示，所以还需要改变列名和行名

![peak-cell matrix](img/atac_matrix.png)

In [ ]:
# 得到染色体位置，到时候直接读入作为行名
awk '{ print $1 ":" $2 "-" $3}' test0.bed > test0.txt
# 删除染色体位置的信息
cut -f 4- test0.bed > test.txt

## scRNA-seq fastq处理

### 下载原始文件、质控、比对

运行脚本

In [ ]:
#!/bin/bash

files=''
for line in `cat SRR_Acc_List.txt`    # SRR_Acc_List文件包含了每个细胞初始数据的SRR号
do
    # 下载SRA 
    prefetch $line
    cd $line
    # 将SRA文件拆解为fastq文件
    fasterq-dump -p --include-technical -S -e 10 -O ./ ${line}.sra
    rm ${line}.sra
    # fastp质控
    fastp -i ${line}_1.fastq -o ${line}_1.clean.fastq -I ${line}_2.fastq -O ${line}_2.clean.fastq
    rm ${line}_1.fastq
    rm ${line}_2.fastq

    # 比对
    STAR --runThreadN 10 --genomeDir /sibcb1/chenluonanlab6/zhangchuanchao2/data/star_index --readFilesIn ${line}_1.clean.fastq ${line}_2.clean.fastq --outFileNamePrefix ${line}_ --outSAMtype BAM SortedByCoordinate --outSAMstrandField intronMotif --outFilterMultimapNmax 2 --outFilterMismatchNmax 3
    rm ${line}_1.clean.fastq
    rm ${line}_2.clean.fastq

    # 使用cuffquant定量分析每个样本的基因转录本表达水平，得到cxb文件
    /sibcb1/chenluonanlab6/zhangchuanchao2/install/cufflinks-2.2.1.Linux_x86_64/cuffquant -o ./. -u -p 10 --library-type fr-unstranded -b /sibcb1/chenluonanlab6/zhangchuanchao2/data/Homo_sapiens/UCSC/hg38/Sequence/WholeGenomeFasta/genome.fa /sibcb1/chenluonanlab6/zhangchuanchao2/data/Homo_sapiens/UCSC/hg38/Annotation/Archives/archive-2015-08-14-08-18-15/Genes/genes.gtf ${line}_Aligned.sortedByCoord.out.bam
    
    # 为后续cuffnorm编写命令中需要的参数
    files=${files}" "${line}"/abundances.cxb"
    cd ..
done

/sibcb1/chenluonanlab6/zhangchuanchao2/install/cufflinks-2.2.1.Linux_x86_64/cuffnorm -o ./. -L sum_fpkm -p 10 --library-norm-method classic-fpkm --library-type fr-unstranded /sibcb1/chenluonanlab6/zhangchuanchao2/data/Homo_sapiens/UCSC/hg38/Annotation/Archives/archive-2015-08-14-08-18-15/Genes/genes.gtf ${files}

最终得到的结果文件中，genes.fpkm_table即是我们需要的gene-cell matrix，修改列名后的文件如下图所示：

![gene-cell matrix](img/rna_matrix.png)